In [34]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [35]:
import os
import tensorflow as tf

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))


column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

TensorFlow version: 2.2.0
Eager execution: True
Local copy of the dataset file: /home/server1/.keras/datasets/iris_training.csv
Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species


In [47]:
batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(train_dataset_fp,batch_size,column_names=column_names,
    label_name=label_name,num_epochs=1)

features, labels = next(iter(train_dataset))

print(features)
print(labels)

OrderedDict([('sepal_length', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([5.4, 6.4, 6.4, 5.7, 6. , 5.1, 6.3, 5.2, 6.4, 6. , 5. , 5.8, 5.3,
       4.8, 4.5, 6.1, 4.8, 5.5, 5.4, 5. , 6.7, 5.7, 6.3, 4.4, 5.9, 5.2,
       4.4, 5. , 5.1, 7.9, 4.9, 4.9], dtype=float32)>), ('sepal_width', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([3.9, 2.8, 3.1, 3.8, 3. , 3.8, 3.3, 2.7, 2.8, 2.2, 3.4, 2.8, 3.7,
       3.4, 2.3, 3. , 3. , 2.6, 3. , 3. , 3.1, 3. , 3.3, 3. , 3.2, 3.4,
       3.2, 3.5, 2.5, 3.8, 3.1, 3.1], dtype=float32)>), ('petal_length', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([1.3, 5.6, 5.5, 1.7, 4.8, 1.6, 4.7, 3.9, 5.6, 5. , 1.6, 5.1, 1.5,
       1.6, 1.3, 4.9, 1.4, 4.4, 4.5, 1.6, 4.4, 4.2, 6. , 1.3, 4.8, 1.4,
       1.3, 1.3, 3. , 6.4, 1.5, 1.5], dtype=float32)>), ('petal_width', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0.4, 2.2, 1.8, 0.3, 1.8, 0.2, 1.6, 1.4, 2.1, 1.5, 0.4, 2.4, 0.2,
       0.2, 0.3, 1.8, 0.3, 1.2, 1.5, 0.2, 1.4, 1.2, 2.5, 0.2,

In [37]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

features, labels=pack_features_vector(features, labels)
print(features)
print(labels)

tf.Tensor(
[[6.5 3.  5.5 1.8]
 [5.4 3.9 1.3 0.4]
 [5.5 2.6 4.4 1.2]
 [4.6 3.6 1.  0.2]
 [6.4 3.2 4.5 1.5]
 [7.7 3.  6.1 2.3]
 [6.  2.9 4.5 1.5]
 [4.4 3.  1.3 0.2]
 [5.4 3.9 1.7 0.4]
 [5.1 3.5 1.4 0.3]
 [6.3 2.7 4.9 1.8]
 [4.8 3.4 1.6 0.2]
 [5.1 3.8 1.6 0.2]
 [7.9 3.8 6.4 2. ]
 [5.8 4.  1.2 0.2]
 [7.2 3.  5.8 1.6]
 [6.2 3.4 5.4 2.3]
 [4.6 3.2 1.4 0.2]
 [4.6 3.1 1.5 0.2]
 [5.1 3.8 1.9 0.4]
 [7.7 3.8 6.7 2.2]
 [4.9 3.  1.4 0.2]
 [6.5 2.8 4.6 1.5]
 [5.2 3.5 1.5 0.2]
 [4.7 3.2 1.3 0.2]
 [4.8 3.  1.4 0.1]
 [6.1 2.9 4.7 1.4]
 [6.3 3.3 6.  2.5]
 [5.  3.4 1.6 0.4]
 [5.5 3.5 1.3 0.2]
 [6.3 2.3 4.4 1.3]
 [5.  3.  1.6 0.2]], shape=(32, 4), dtype=float32)
tf.Tensor([2 0 1 0 1 2 1 0 0 0 2 0 0 2 0 2 2 0 0 0 2 0 1 0 0 0 1 2 0 0 1 0], shape=(32,), dtype=int32)


In [38]:
train_dataset = train_dataset.map(pack_features_vector)

In [39]:
features, labels = next(iter(train_dataset))

In [41]:

model = tf.keras.Sequential([tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),tf.keras.layers.Dense(3)])

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)

  return loss_object(y_true=y, y_pred=y_)


l = loss(model, features, labels, training=False)
print("Loss test: {}".format(l))


def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)



optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels, training=True).numpy()))


## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 2001

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    epoch_accuracy.update_state(y, model(x, training=True))

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))


Loss test: 2.3085553646087646
Step: 0, Initial Loss: 2.3085553646087646
Step: 1,         Loss: 1.646769404411316
Epoch 000: Loss: 1.311, Accuracy: 30.000%
Epoch 050: Loss: 0.293, Accuracy: 95.000%
Epoch 100: Loss: 0.214, Accuracy: 97.500%
Epoch 150: Loss: 0.166, Accuracy: 97.500%
Epoch 200: Loss: 0.139, Accuracy: 97.500%
Epoch 250: Loss: 0.119, Accuracy: 97.500%
Epoch 300: Loss: 0.101, Accuracy: 97.500%
Epoch 350: Loss: 0.100, Accuracy: 97.500%
Epoch 400: Loss: 0.094, Accuracy: 97.500%
Epoch 450: Loss: 0.103, Accuracy: 97.500%
Epoch 500: Loss: 0.097, Accuracy: 98.333%
Epoch 550: Loss: 0.080, Accuracy: 99.167%
Epoch 600: Loss: 0.082, Accuracy: 97.500%
Epoch 650: Loss: 0.087, Accuracy: 97.500%
Epoch 700: Loss: 0.074, Accuracy: 98.333%
Epoch 750: Loss: 0.081, Accuracy: 98.333%
Epoch 800: Loss: 0.110, Accuracy: 98.333%
Epoch 850: Loss: 0.064, Accuracy: 99.167%
Epoch 900: Loss: 0.067, Accuracy: 96.667%
Epoch 950: Loss: 0.074, Accuracy: 97.500%
Epoch 1000: Loss: 0.078, Accuracy: 98.333%
Epoc

In [45]:
test_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"

test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url),
                                  origin=test_url)


test_dataset = tf.data.experimental.make_csv_dataset(
    test_fp,
    batch_size,
    column_names=column_names,
    label_name='species',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

test_accuracy = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset:
  # training=False is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  logits = model(x, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

Test set accuracy: 96.667%


In [48]:
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']
predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

# training=False is needed only if there are layers with different
# behavior during training versus inference (e.g. Dropout).
predictions = model(predict_dataset, training=False)

for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))

Example 0 prediction: Iris setosa (99.9%)
Example 1 prediction: Iris versicolor (99.9%)
Example 2 prediction: Iris virginica (94.2%)
